In [12]:
!pip install music21

In [13]:
!pip install pygame

In [14]:
!pip install tensorflow music21 pandas pygame mido

In [15]:
# Step 1: Importing necessary libraries
import os
import numpy as np
import pandas as pd
import music21
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, LSTM, Dropout, Input
from tensorflow.keras.optimizers import Adam
import pygame

In [16]:
# from music21 import midi
# from music21 import converter, instrument  # or from music21 import *

# # Create a list to hold all the paths to the MIDI files
# midi_paths = []

# # Traverse through the 'lakh-midi-clean' dataset directory and collect all MIDI file paths
# for dirname, _, filenames in os.walk('/kaggle/input/lakh-midi-clean'):
#     for filename in filenames:
#         if filename.endswith(".mid"):  # check if the file is a MIDI file
#             midi_paths.append(os.path.join(dirname, filename))

# # Display the number of MIDI files found
# print(f"Found {len(midi_paths)} MIDI files.")

# # Now, let's load and explore one of these MIDI files using music21
# sample_midi_path = midi_paths[0]  # let's use the first MIDI file as a sample
# sample_midi_file = converter.parse(sample_midi_path)  # parsing the MIDI file

# # Print some information about the sample MIDI file
# print("Sample MIDI File:", sample_midi_path)

# # Getting all instruments
# all_instruments = instrument.partitionByInstrument(sample_midi_file)

# if all_instruments:  # if there are any instruments available
#     print("Number of instruments:", len(all_instruments.parts))
#     for i, part in enumerate(all_instruments.parts):
#         inst = part.getInstrument() or "No Instrument"
#         print(f"- Instrument {i}: {inst}")
# else:
#     print("No instruments found.")

In [17]:
# Instalación de FluidSynth y dependencias
!apt-get update -y
!apt-get install -y fluidsynth

# Descarga de una SoundFont
!wget https://github.com/FluidSynth/fluidsynth/files/3443135/default_sound_font.sf2.tar.gz
!tar xzvf default_sound_font.sf2.tar.gz

# Instalación de midi2audio
!pip install midi2audio

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
--2023-11-06 10:39:32--  https://github.com/FluidSynth/fluidsynth/files/3443135/default_sound_font.sf2.tar.gz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-11-06 10:39:33 ERROR 404: Not Found.

tar (child): default_sound_font.sf2.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: 

In [18]:
# from midi2audio import FluidSynth

# # Ruta al archivo MIDI que deseas convertir
# input_midi = '/kaggle/input/lakh-midi-clean/AC_DC/Back_In_Black.1.mid'

# # Ruta al archivo WAV de salida donde se guardará el audio
# output_wav = 'output_audio.wav'

# # Ruta a tu archivo SoundFont
# soundfont = 'default_sound_font.sf2'

# # Inicializar FluidSynth con la SoundFont especificada
# fs = FluidSynth(sound_font=soundfont)

# # Convertir el archivo MIDI a WAV
# fs.midi_to_audio(input_midi, output_wav)

# print(f'Archivo MIDI convertido a WAV y guardado en: {output_wav}')

In [19]:
# from IPython.display import Audio

# # Ruta al archivo WAV de salida que quieres reproducir
# output_wav = 'output_audio.wav'

# # Reproduce el archivo de audio en el notebook
# Audio(output_wav)

In [20]:
# import pickle
# import os
# from tqdm import tqdm
# from music21 import converter, instrument, note, chord
# import threading

# from collections import deque

# SEQUENCE_LENGTH = 100  # Tamaño de la secuencia

# # Función para obtener secuencias a partir de una lista de notas
# def create_sequences(notes_list):
#     input_sequences = []
#     output_notes = []

#     for i in range(len(notes_list) - SEQUENCE_LENGTH):
#         sequence_in = notes_list[i:i + SEQUENCE_LENGTH]
#         sequence_out = notes_list[i + SEQUENCE_LENGTH]
        
#         input_sequences.append(sequence_in)
#         output_notes.append(sequence_out)

#     return input_sequences, output_notes


# # Función para procesar un archivo MIDI y obtener las notas
# def get_notes_from_midi(midi_path, event, local_notes):
#     midi_file = converter.parse(midi_path)
    
#     try:  # archivo tiene instrumentos
#         s2 = instrument.partitionByInstrument(midi_file)
#         notes_to_parse = s2.parts[0].recurse() 
#     except:  # archivo no tiene instrumentos
#         notes_to_parse = midi_file.flat.notes

#     for element in notes_to_parse:
#         if isinstance(element, note.Note):
#             local_notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             local_notes.append('.'.join(str(n) for n in element.normalOrder))
    
#     event.set()

# # Función para dividir la lista de archivos MIDI en grupos
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]

# # Número de archivos por grupo
# CHUNK_SIZE = 1000

# # Directorio para guardar los archivos
# OUTPUT_DIR = 'processed_chunks'
# if not os.path.exists(OUTPUT_DIR):
#     os.makedirs(OUTPUT_DIR)

# # Divide la lista de archivos MIDI en grupos
# midi_chunks = list(chunks(midi_paths, CHUNK_SIZE))

# # Procesa cada grupo
# for index, midi_chunk in enumerate(midi_chunks, 1):
#     chunk_file_path = os.path.join(OUTPUT_DIR, f'sequences_chunk_{index}.pkl')
    
#     # Si el archivo ya existe, significa que este grupo ya fue procesado
#     if os.path.exists(chunk_file_path):
#         print(f"Chunk {index} ya procesado. Saltando...")
#         continue
    
#     # Procesa el grupo
#     all_notes = []
#     for midi_path in tqdm(midi_chunk, desc=f"Processing chunk {index}", unit="file"):
#         local_notes = []
#         event = threading.Event()
        
#         thread = threading.Thread(target=get_notes_from_midi, args=(midi_path, event, local_notes))
#         thread.start()
        
#         # Si el hilo no completa en 20 segundos, se omite el archivo
#         thread.join(timeout=20)
#         if not event.is_set():
#             print(f"Tiempo límite alcanzado para el archivo {midi_path}. Saltando este archivo.")
#             continue
        
#         all_notes.extend(local_notes)

#     # Convertir las notas en secuencias de entrada y salida
#     input_sequences, output_notes = create_sequences(all_notes)

#     # Guarda las secuencias y las notas de salida
#     with open(chunk_file_path, 'wb') as file:
#         pickle.dump((input_sequences, output_notes), file)
#         print(f"Secuencias del chunk {index} guardadas en {chunk_file_path}")


In [22]:
import os
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

SEQUENCE_LENGTH = 100  # Tamaño de la secuencia


OUTPUT_DIR = 'processed_chunks'
input_sequences = []
output_notes = []

# Cargar todos los archivos de secuencias en el directorio de chunks
for chunk_filename in os.listdir(OUTPUT_DIR):
    if chunk_filename.startswith("sequences_chunk_") and chunk_filename.endswith(".pkl"):
        try:
            with open(os.path.join(OUTPUT_DIR, chunk_filename), 'rb') as file:
                sequences_chunk = pickle.load(file)
                input_sequences.extend(sequences_chunk[0])
                output_notes.extend(sequences_chunk[1])
        except EOFError as e:
            print(f"No se pudo cargar {chunk_filename}: {e}")


# Ahora tenemos todas las secuencias de entrada y notas de salida, construyamos el mapeo de notas a enteros.
all_notes = set(note for seq in input_sequences for note in seq) | set(output_notes)
pitchnames = sorted(all_notes)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# convertir las secuencias de notas y las notas de salida en números enteros
for seq in input_sequences:
    network_input.append([note_to_int[note] for note in seq])
network_output = [note_to_int[note] for note in output_notes]

n_patterns = len(network_input)

# remodelar la entrada para ser compatible con las capas LSTM y normalizar
network_input = np.reshape(network_input, (n_patterns, SEQUENCE_LENGTH, 1))
network_input = network_input / float(len(pitchnames))

# convertir la salida en un formato categórico
network_output = to_categorical(network_output)

print(network_input.shape)
n_vocab = len(pitchnames)
print(f"Número de notas únicas: {n_vocab}")


No se pudo cargar sequences_chunk_11.pkl: Ran out of input
(2443813, 100, 1)
Número de notas únicas: 1269


In [ ]:
# def create_discriminator():
#     model = Sequential()
#     model.add(LSTM(512, return_sequences=True, input_shape=(100, 853)))  # Aceptar la salida del generador
#     model.add(Dropout(0.3))
#     model.add(LSTM(512))
#     model.add(Dropout(0.3))
#     model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam')
#     return model

In [ ]:
# def create_generator():
#     model = Sequential()
#     model.add(LSTM(512, return_sequences=True, input_shape=(sequence_length, 1)))  
#     model.add(Dropout(0.3))
#     model.add(LSTM(512, return_sequences=True))
#     model.add(Dropout(0.3))
#     model.add(LSTM(512, return_sequences=True))  # Haz que esta capa LSTM también devuelva secuencias
#     model.add(Dense(n_vocab, activation='softmax'))  # Esta capa ahora producirá una distribución de probabilidad para cada paso de la secuencia
#     return model

In [24]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input, TimeDistributed
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.utils import to_categorical

In [25]:
sequence_length = 100  # Asumiendo que cada secuencia tiene 100 pasos de tiempo

def create_discriminator(input_shape=(100, 1269)):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, activation='tanh', input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(LSTM(128, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

In [26]:
def create_generator(sequence_length=100, n_vocab=1269):
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, activation='tanh', input_shape=(sequence_length, n_vocab)))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(LSTM(128, return_sequences=True, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(TimeDistributed(Dense(n_vocab, activation='softmax')))
    return model


In [27]:
def create_gan(discriminator, generator, sequence_length, n_vocab):
    # Asegúrate de que el discriminador no sea entrenable para que no se actualice durante el proceso de entrenamiento del GAN.
    discriminator.trainable = False
    
    # La entrada al GAN debe tener la misma forma que la salida esperada del generador.
    gan_input = Input(shape=(sequence_length, n_vocab))
    
    # Genera secuencias con el generador usando la entrada del GAN.
    x = generator(gan_input)
    
    # Obtiene la salida del discriminador usando las secuencias generadas.
    gan_output = discriminator(x)
    
    # Crea el modelo GAN combinando el generador y el discriminador.
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

In [28]:
# Crear los modelos
discriminator = create_discriminator()
generator = create_generator(sequence_length, n_vocab)
gan = create_gan(discriminator, generator, sequence_length, n_vocab)

2023-11-06 10:45:21.725251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-06 10:45:21.797130: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [29]:
def integer_to_onehot(data, vocab_size=853):
    """
    Convertir una secuencia de enteros a representación one-hot.
    """
    onehot = np.zeros((data.shape[0], data.shape[1], vocab_size))
    for i, sequence in enumerate(data):
        for j, note in enumerate(sequence):
            onehot[i, j, int(note)] = 1
    return onehot


In [31]:
import tensorflow as tf

# List all available devices
print(tf.config.list_physical_devices())

# Specifically check for a GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Num GPUs Available:  0


In [ ]:
epochs = 1000  # número de épocas para entrenar, se debe ajustar
batch_size = 32  # tamaño de los lotes de entrenamiento, se debe ajustar

# Crear datos de entrenamiento para el generador con ruido.
# Este ruido servirá como entrada para el generador.
noise = np.random.normal(0, 1, (batch_size, sequence_length, n_vocab))

for epoch in range(epochs):
    # ---------------------
    #  Entrenar Discriminador
    # ---------------------

    # Seleccionar un conjunto aleatorio de secuencias de notas
    idx = np.random.randint(0, network_input.shape[0], batch_size)
    real_seqs_onehot = integer_to_onehot(network_input[idx], vocab_size=n_vocab)

    # Generar un conjunto de nuevas secuencias de notas (usando 'ruido').
    gen_seqs = generator.predict(noise)
    
    # Entrenar el discriminador
    d_loss_real = discriminator.train_on_batch(real_seqs_onehot, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_seqs, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Entrenar Generador
    # ---------------------

    noise = np.random.normal(0, 1, (batch_size, sequence_length, n_vocab))

    # Entrenar el generador (el discriminador está 'congelado' en este entrenamiento)
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    # Mostrar el progreso
    print(f"{epoch}/{epochs} [D loss: {d_loss}] [G loss: {g_loss}]")


1/1 [==============================] - 0s 485ms/step
0/1000 [D loss: 0.0010261503921356052] [G loss: 7.471803665161133]
1/1 [==============================] - 0s 443ms/step
1/1000 [D loss: 0.0007406558725051582] [G loss: 7.4758195877075195]
1/1 [==============================] - 0s 446ms/step
2/1000 [D loss: 0.000692218542098999] [G loss: 7.781186580657959]
1/1 [==============================] - 0s 446ms/step
3/1000 [D loss: 0.0006082087056711316] [G loss: 7.749504089355469]
1/1 [==============================] - 0s 477ms/step
4/1000 [D loss: 0.0005298921896610409] [G loss: 7.819937705993652]
1/1 [==============================] - 0s 444ms/step
5/1000 [D loss: 0.0004642036510631442] [G loss: 7.959303855895996]
1/1 [==============================] - 0s 453ms/step
6/1000 [D loss: 0.00043305946746841073] [G loss: 8.108146667480469]
1/1 [==============================] - 0s 444ms/step
7/1000 [D loss: 0.00041227317706216127] [G loss: 8.25255012512207]
1/1 [==============================] - 

In [ ]:
# Guardar el modelo del generador
generator.save('generator_model.h5')

# Guardar el modelo del discriminador
discriminator.save('discriminator_model.h5')

In [ ]:
from keras.models import load_model

# Cargar el modelo del generador
loaded_generator = load_model('generator_model.h5')

# Cargar el modelo del discriminador
loaded_discriminator = load_model('discriminator_model.h5')

In [ ]:
noise = np.random.normal(0, 1, [1, 100])
generated_seq = loaded_generator.predict(noise)  # Usamos 'loaded_generator' aquí

generated_notes = onehot_to_integer(generated_seq[0])

In [ ]:
generated_notes = onehot_to_integer(generated_seq[0])

In [ ]:
def onehot_to_integer(onehot_seq):
    return [np.argmax(note) for note in onehot_seq]

In [ ]:
!pip install mido

In [ ]:
from mido import Message, MidiFile, MidiTrack

midi = MidiFile()
track = MidiTrack()
midi.tracks.append(track)

for note in generated_notes:
    track.append(Message('note_on', note=note, velocity=64, time=32))
    track.append(Message('note_off', note=note, velocity=64, time=32))

midi.save('generated_song.mid')

In [ ]:
from midi2audio import FluidSynth

# Ruta al archivo MIDI que deseas convertir
input_midi = '/kaggle/working/generated_song.mid'

# Ruta al archivo WAV de salida donde se guardará el audio
output_wav = 'generated_song.wav'

# Ruta a tu archivo SoundFont
soundfont = 'default_sound_font.sf2'

# Inicializar FluidSynth con la SoundFont especificada
fs = FluidSynth(sound_font=soundfont)

# Convertir el archivo MIDI a WAV
fs.midi_to_audio(input_midi, output_wav)

print(f'Archivo MIDI convertido a WAV y guardado en: {output_wav}')

In [ ]:
from IPython.display import Audio

# Ruta al archivo WAV de salida que quieres reproducir
output_wav = 'generated_song.wav'

# Reproduce el archivo de audio en el notebook
Audio(output_wav)